## Phase 1

In [3]:
import json
import math
import re
from collections import Counter
from collections import defaultdict

from hazm import Stemmer

f = open('IR_data_news_12k.json')
# f = open('IR_data_news_5k 2.json')
json_data = json.load(f)
contents, titles, urls = zip(*[(item.get("content", ""), item.get("title", ""),
                                item.get("url", "")) for item in json_data.values()])

# Example:
print('1st doc:')
print("Title:", titles[0])
print("URL:", urls[0])
print("Content:", contents[0])


1st doc:
Title: اعلام زمان قرعه کشی جام باشگاه های فوتسال آسیا
URL: https://www.farsnews.ir/news/14001224001005/اعلام-زمان-قرعه-کشی-جام-باشگاه-های-فوتسال-آسیا
Content: 
به گزارش خبرگزاری فارس، کنفدراسیون فوتبال آسیا (AFC) در نامه ای رسمی به فدراسیون فوتبال ایران و باشگاه گیتی پسند زمان  قرعه کشی جام باشگاه های فوتسال آسیا را رسماً اعلام کرد. بر این اساس 25 فروردین ماه 1401 مراسم قرعه کشی جام باشگاه های فوتسال آسیا در مالزی برگزار می شود. باشگاه گیتی پسند بعنوان قهرمان فوتسال ایران در سال 1400 به این مسابقات راه پیدا کرده است. پیش از این گیتی پسند تجربه 3 دوره حضور در جام باشگاه های فوتسال آسیا را داشته که هر سه دوره به فینال مسابقات راه پیدا کرده و یک عنوان قهرمانی و دو مقام دومی بدست آورده است. انتهای پیام/





In [4]:
# lists and parameters used for preprocessing
verbs = []
punctuations_replacements = []
diacritics_replacements = []
specials_chars_replacements = []
unicodes_replacements = []
punc_before = None
punc_after = None
spacing_patterns = []
punctuation_spacing_replacements = []
# -------------------------------------------------

# lists and parameters initializing
with open('verbs.txt', 'r', encoding='utf-8') as verbs_file:
    for v in verbs_file:
        verbs.append(v[:-1])

punctuations_replacements = [
    ("[!\"\#\$\%\&\'\(\)\*\+\,\-\.\/\:\;\<\=\>\?\؟\@\[\]\^\_\`\؛\،\{\|\}\~\«\»\…\ْ\ٌ\ٍ\ً\ُ\ِ\َ\ّ\ء\ٔ\ٰ\﷼]", "")]

diacritics_replacements = [
    # remove FATHATAN, DAMMATAN, KASRATAN, FATHA, DAMMA, KASRA, SHADDA, SUKUN
    ("[\u064b\u064c\u064d\u064e\u064f\u0650\u0651\u0652]", ""),
]

specials_chars_replacements = [
    # Remove almoast all arabic unicode superscript and subscript characters in the ranges of 00600-06FF, 08A0-08FF, FB50-FDFF, and FE70-FEFF
    (
        "[\u0605\u0653\u0654\u0655\u0656\u0657\u0658\u0659\u065a\u065b\u065c\u065d\u065e\u065f\u0670\u0610\u0611\u0612\u0613\u0614\u0615\u0616\u0618\u0619\u061a\u061e\u06d4\u06d6\u06d7\u06d8\u06d9\u06da\u06db\u06dc\u06dd\u06de\u06df\u06e0\u06e1\u06e2\u06e3\u06e4\u06e5\u06e6\u06e7\u06e8\u06e9\u06ea\u06eb\u06ec\u06ed\u06fd\u06fe\u08ad\u08d4\u08d5\u08d6\u08d7\u08d8\u08d9\u08da\u08db\u08dc\u08dd\u08de\u08df\u08e0\u08e1\u08e2\u08e3\u08e4\u08e5\u08e6\u08e7\u08e8\u08e9\u08ea\u08eb\u08ec\u08ed\u08ee\u08ef\u08f0\u08f1\u08f2\u08f3\u08f4\u08f5\u08f6\u08f7\u08f8\u08f9\u08fa\u08fb\u08fc\u08fd\u08fe\u08ff\ufbb2\ufbb3\ufbb4\ufbb5\ufbb6\ufbb7\ufbb8\ufbb9\ufbba\ufbbb\ufbbc\ufbbd\ufbbe\ufbbf\ufbc0\ufbc1\ufc5e\ufc5f\ufc60\ufc61\ufc62\ufc63\ufcf2\ufcf3\ufcf4\ufd3e\ufd3f\ufe70\ufe71\ufe72\ufe76\ufe77\ufe78\ufe79\ufe7a\ufe7b\ufe7c\ufe7d\ufe7e\ufe7f\ufdfa\ufdfb]",
        "",
    ),
]

unicodes_replacements = [
    ("ك", 'ک'),
    ("ي", "ی"),
    ("آ|أ", "ا"),
    ("﷽", "بسم الله الرحمن الرحیم"),
    ("﷼", "ریال"),
    ("(ﷰ|ﷹ)", "صلی"),
    ("ﷲ", "الله"),
    ("ﷳ", "اکبر"),
    ("ﷴ", "محمد"),
    ("ﷵ", "صلعم"),
    ("ﷶ", "رسول"),
    ("ﷷ", "علیه"),
    ("ﷸ", "وسلم"),
    ("ﻵ|ﻶ|ﻷ|ﻸ|ﻹ|ﻺ|ﻻ|ﻼ", "لا"),
]

punc_after, punc_before = r"\.:!،؛؟»\]\)\}", r"«\[\(\{"

spacing_patterns = [
    (r"([^ ]ه) ی ", r"\1‌ی "),  # fix ی space
    (r"(^| )(ن?می) ", r"\1\2‌"),  # put zwnj after می, نمی
    # put zwnj before تر, تری, ترین, گر, گری, ها, های, هایی
    (
        r"(?<=[^\n\d "
        + punc_after
        + punc_before
        + "]{2}) (تر(ین?)?|گری?|های?|هایی?)(?=[ \n"
        + punc_after
        + punc_before
        + "]|$)",
        r"‌\1",
    ),
    # join ام, ایم, اش, اند, ای, اید, ات
    (
        r"([^ ]ه) (ا(م|یم|ش|ند|ی|ید|ت))(?=[ \n" + punc_after + "]|$)",
        r"\1‌\2",
    ),
    # شنبهها => شنبه‌ها
    ("(ه)(ها)", r"\1‌\2"),
]

extra_space_replacements = [
    (r" {2,}", " "),  # remove extra spaces
    (r"\n{3,}", "\n\n"),  # remove extra newlines
    (r"\u200c{2,}", "\u200c"),  # remove extra ZWNJs
    (r"\u200c{1,} ", " "),  # remove unneded ZWNJs before space
    (r" \u200c{1,}", " "),  # remove unneded ZWNJs after space
    (r"\b\u200c*\B", ""),  # remove unneded ZWNJs at the beginning of words
    (r"\B\u200c*\b", ""),  # remove unneded ZWNJs at the end of words
    (r"[ـ\r]", ""),  # remove keshide, carriage returns
]

punctuation_spacing_replacements = [
    # remove space before and after quotation
    ('" ([^\n"]+) "', r'"\1"'),
    (" ([" + punc_after + "])", r"\1"),  # remove space before
    ("([" + punc_before + "]) ", r"\1"),  # remove space after
    # put space after . and :
    (
        "([" + punc_after[:3] + "])([^ " + punc_after + r"\d۰۱۲۳۴۵۶۷۸۹])",
        r"\1 \2",
    ),
    (
        "([" + punc_after[3:] + "])([^ " + punc_after + "])",
        r"\1 \2",
    ),  # put space after
    (
        "([^ " + punc_before + "])([" + punc_before + "])",
        r"\1 \2",
    ),  # put space before
    # put space after number; e.g., به طول ۹متر -> به طول ۹ متر
    (r"(\d)([آابپتثجچحخدذرزژسشصضطظعغفقکگلمنوهی])", r"\1 \2"),
    # put space after number; e.g., به طول۹ -> به طول ۹
    (r"([آابپتثجچحخدذرزژسشصضطظعغفقکگلمنوهی])(\d)", r"\1 \2"),
]

number_replacements = {
    ('0', '۰'),
    ('1', '۱'),
    ('2', '۲'),
    ('3', '۳'),
    ('4', '۴'),
    ('5', '۵'),
    ('6', '۶'),
    ('7', '۷'),
    ('8', '۸'),
    ('9', '۹'),
    ('%', '٪')
}

In [5]:
def replace(patterns, text):
    for pattern, repl in patterns:
        text = re.sub(pattern, repl, text)
    return text


def separate_mi(text):
    words = text.split()
    for i in range(len(words)):
        if re.match(r"\bن?می[آابپتثجچحخدذرزژسشصضطظعغفقکگلمنوهی]+", words[i]):
            prefix = re.sub("^(ن?می)", r"\1‌", words[i])
            if prefix in verbs:
                words[i] = prefix
    return ' '.join(words)


def correct_spacing(text):
    text = replace(extra_space_replacements, text)
    text = replace(punctuation_spacing_replacements, text)
    return replace(spacing_patterns, text)


def normalize(text):
    text = replace(punctuations_replacements, text)
    text = replace(specials_chars_replacements, text)
    text = replace(diacritics_replacements, text)
    text = replace(unicodes_replacements, text)
    text = separate_mi(text)
    text = correct_spacing(text)
    text = replace(number_replacements, text)

    return text

# preprocessing test
texts = []
texts.append('0123456789%٠١٢٣٤٥٦٧٨٩')
texts.append('پیامبر اکرم ﷺ')
texts.append('حَذفِ اِعراب')
texts.append('به          طول ۹متر و عرض۶')
texts.append('سلام دنیـــا می ایم میروم')
texts.append('جمعهها که کار نمیکنم مطالعه میکنم')
texts.append('')
texts.append('')
for text in texts:
    print(normalize(text))

۰۱۲۳۴۵۶۷۸۹٠١٢٣٤٥٦٧٨٩
پیامبر اکرم
حذف اعراب
به طول ۹ متر و عرض ۶
سلام دنیا می‌ایم می‌روم
جمعه‌ها که کار نمی‌کنم مطالعه می‌کنم




In [6]:
after_verbs = {
    "ام",
    "ای",
    "است",
    "ایم",
    "اید",
    "اند",
    "بودم",
    "بودی",
    "بود",
    "بودیم",
    "بودید",
    "بودند",
    "باشم",
    "باشی",
    "باشد",
    "باشیم",
    "باشید",
    "باشند",
    "شده_ام",
    "شده_ای",
    "شده_است",
    "شده_ایم",
    "شده_اید",
    "شده_اند",
    "شده_بودم",
    "شده_بودی",
    "شده_بود",
    "شده_بودیم",
    "شده_بودید",
    "شده_بودند",
    "شده_باشم",
    "شده_باشی",
    "شده_باشد",
    "شده_باشیم",
    "شده_باشید",
    "شده_باشند",
    "نشده_ام",
    "نشده_ای",
    "نشده_است",
    "نشده_ایم",
    "نشده_اید",
    "نشده_اند",
    "نشده_بودم",
    "نشده_بودی",
    "نشده_بود",
    "نشده_بودیم",
    "نشده_بودید",
    "نشده_بودند",
    "نشده_باشم",
    "نشده_باشی",
    "نشده_باشد",
    "نشده_باشیم",
    "نشده_باشید",
    "نشده_باشند",
    "شوم",
    "شوی",
    "شود",
    "شویم",
    "شوید",
    "شوند",
    "شدم",
    "شدی",
    "شد",
    "شدیم",
    "شدید",
    "شدند",
    "نشوم",
    "نشوی",
    "نشود",
    "نشویم",
    "نشوید",
    "نشوند",
    "نشدم",
    "نشدی",
    "نشد",
    "نشدیم",
    "نشدید",
    "نشدند",
    "می‌شوم",
    "می‌شوی",
    "می‌شود",
    "می‌شویم",
    "می‌شوید",
    "می‌شوند",
    "می‌شدم",
    "می‌شدی",
    "می‌شد",
    "می‌شدیم",
    "می‌شدید",
    "می‌شدند",
    "نمی‌شوم",
    "نمی‌شوی",
    "نمی‌شود",
    "نمی‌شویم",
    "نمی‌شوید",
    "نمی‌شوند",
    "نمی‌شدم",
    "نمی‌شدی",
    "نمی‌شد",
    "نمی‌شدیم",
    "نمی‌شدید",
    "نمی‌شدند",
    "خواهم_شد",
    "خواهی_شد",
    "خواهد_شد",
    "خواهیم_شد",
    "خواهید_شد",
    "خواهند_شد",
    "نخواهم_شد",
    "نخواهی_شد",
    "نخواهد_شد",
    "نخواهیم_شد",
    "نخواهید_شد",
    "نخواهند_شد",
}

before_verbs = {
    "خواهم",
    "خواهی",
    "خواهد",
    "خواهیم",
    "خواهید",
    "خواهند",
    "نخواهم",
    "نخواهی",
    "نخواهد",
    "نخواهیم",
    "نخواهید",
    "نخواهند",
}

verbe = set(
    [verb + "ه" for verb in verbs]
    + ["ن" + verb + "ه" for verb in verbs],
)


def tokenize(text):
    pattern = re.compile(r'([؟!?]+|[\d.:]+|[:.،؛»\])}"«\[({/\\])')
    text = pattern.sub(r" \1 ", text.replace("\n", " ").replace("\t", " "))
    tokens = [word for word in text.split(" ") if word]
    tokens = join_verb_parts(tokens)

    return tokens


def join_verb_parts(tokens):
    if len(tokens) == 1:
        return tokens

    result = [""]
    for token in reversed(tokens):
        if token in before_verbs or (
                result[-1] in after_verbs and token in verbe
        ):
            result[-1] = token + "_" + result[-1]
        else:
            result.append(token)
    return list(reversed(result[1:]))


In [7]:
def generate_tokens(contents):
    overall_tokens = []
    stemmer = Stemmer()
    for c in contents:
        c = normalize(c)
        tokens = tokenize(c)
        for i in range(len(tokens)):
            tokens[i] = stemmer.stem(tokens[i])
        overall_tokens.append(tokens)
    return overall_tokens


docs = generate_tokens(contents)

In [8]:
def remove_most_frequent_tokens(docs):
    flattened_list = [item for sublist in docs for item in sublist]
    item_frequencies = Counter(flattened_list)
    sorted_frequent = sorted(item_frequencies.items(), key=lambda x: x[1], reverse=True)[:50]
    print('> frequent_terms:')
    for frq in sorted_frequent:
        print(frq)
    frequent_terms = [x for x, _ in sorted_frequent]
    print(frequent_terms)
    new_docs=[]
    for doc in docs:
        dd=[]
        for t in doc:
            if not t in frequent_terms:
                dd.append(t)
        new_docs.append(dd)
    return new_docs


docs = remove_most_frequent_tokens(docs)


> frequent_terms:
('و', 234741)
('در', 165012)
('به', 136382)
('از', 92924)
('این', 84204)
('که', 76243)
('را', 69659)
('با', 69230)
('اس', 46515)
('برا', 30975)
('ه', 28174)
('کرد', 26725)
('یک', 23902)
('کشور', 22369)
('ت', 22289)
('ما', 19729)
('خود', 18859)
('بر', 18829)
('', 18579)
('شد', 17098)
('باز', 16821)
('باید', 16197)
('تا', 15919)
('اما', 15197)
('گف', 14751)
('مجلس', 14562)
('اسلام', 14449)
('فارس', 14173)
('مرد', 13944)
('بود', 13234)
('گزار', 13166)
('دو', 12874)
('پیا', 12811)
('کار', 12807)
('ایر', 12728)
('سال', 12638)
('مل', 12464)
('خبرگزار', 12416)
('انت', 12352)
('رئیس', 11628)
('دول', 11589)
('ب', 11212)
('دارد', 10782)
('بازیکن', 10625)
('دا', 10440)
('شود', 10260)
('اینکه', 10052)
('قرار', 9492)
('ان', 9304)
('انقلاب', 9168)
['و', 'در', 'به', 'از', 'این', 'که', 'را', 'با', 'اس', 'برا', 'ه', 'کرد', 'یک', 'کشور', 'ت', 'ما', 'خود', 'بر', '', 'شد', 'باز', 'باید', 'تا', 'اما', 'گف', 'مجلس', 'اسلام', 'فارس', 'مرد', 'بود', 'گزار', 'دو', 'پیا', 'کار', 'ایر', 'سال', '

In [9]:
def generate_positional_index(docs):
    postings = {}
    for doc_id, doc in enumerate(docs):
        for pos, token in enumerate(doc):
            if token not in postings:
                postings[token] = {'doc_freq': 0, 'posting': {}, 'tf_idf': {}}
            if doc_id in postings[token]['posting']:
                postings[token]['posting'][doc_id].append(pos)
            else:
                postings[token]['posting'][doc_id] = [pos]
                postings[token]['doc_freq'] += 1
    return postings


postings = generate_positional_index(docs)

In [10]:
postings['تیم']

{'doc_freq': 2736,
 'posting': {3: [57],
  4: [8],
  8: [286],
  9: [13, 18, 29],
  11: [10],
  14: [57],
  17: [53],
  24: [99],
  28: [113],
  29: [589],
  31: [184, 308],
  35: [72],
  44: [128],
  48: [53, 215, 296, 365],
  49: [27],
  51: [12],
  53: [116, 221, 285, 291, 341, 386, 421, 448, 478, 486, 538],
  54: [39, 97, 108, 199, 236, 360],
  55: [33, 101, 107],
  56: [48],
  58: [150, 171],
  61: [2, 20],
  63: [37],
  65: [21, 26, 32, 157],
  66: [34, 187, 195],
  69: [29, 33, 52],
  70: [19, 167],
  71: [144],
  72: [15, 29],
  73: [19, 34],
  76: [41, 166],
  88: [68, 136],
  91: [137, 151],
  92: [33],
  94: [37, 114],
  99: [49],
  102: [25, 48],
  104: [259],
  107: [23, 48],
  109: [2],
  114: [51],
  115: [18],
  120: [30],
  122: [2],
  126: [45, 102],
  128: [9, 32, 50, 91, 97],
  129: [56, 86, 93],
  139: [4],
  140: [284],
  142: [153, 210, 225, 495, 496],
  143: [119],
  147: [167],
  148: [41, 45, 139, 162, 169, 377],
  149: [72],
  151: [12],
  152: [20, 34, 141],

## Phase 2

In [11]:
def tokenize_query(query):
    placeholders = []
    def repl(m):
        placeholders.append(m.group())
        return f'___{len(placeholders) - 1}___'

    query = re.sub(r'"[^"]*"', repl, query)

    tokens = query.split()

    for i, ph in enumerate(placeholders):
        tokens = [t.replace(f'___{i}___', ph) for t in tokens]

    return tokens


def calculate_query_tf(query):
    query_tf = {}

    for token in tokenize_query(query):
        if token in query_tf.keys():
            query_tf[token] += 1
        else:
            query_tf[token] = 1

    for token in query_tf.keys():
        query_tf[token] = 1 + math.log10(query_tf[token])

    tf_sum = sum(math.pow(tf, 2) for tf in query_tf.values())

    return query_tf, math.sqrt(tf_sum)


def calculate_tf_idf(postings, docs):
    document_length = defaultdict(float)

    for token, posting in postings.items():
        idf = math.log10(len(docs) / posting['doc_freq'])

        for doc_id, positions in posting['posting'].items():
            tf = math.log10(len(positions)) + 1
            tf_idf = tf * idf
            posting['tf_idf'][doc_id] = tf_idf
            document_length[doc_id] += tf_idf ** 2

    for doc_id, length in document_length.items():
        document_length[doc_id] = math.sqrt(length)

    return dict(document_length)


def generate_champions(postings_lists):
    champion_lists = {}
    for term in postings_lists:
        tf_idf_scores = {}
        for doc_id in postings_lists[term]['posting'].keys():
            if doc_id in tf_idf_scores.keys():
                tf_idf_scores[doc_id] += postings_lists[term]['tf_idf'][doc_id] / doc_len[doc_id]
            else:
                tf_idf_scores[doc_id] = postings_lists[term]['tf_idf'][doc_id] / doc_len[doc_id]

        sorted_docs = sorted(tf_idf_scores.items(), key=lambda x: x[1], reverse=True)
        r = min(60, len(sorted_docs))
        champion_lists[term] = [doc_id for doc_id, _ in
                                sorted_docs[:r]]

    return champion_lists


In [12]:

def calculate_jaccard_similarity(query, use_champions=True):
    query_tokens = set(tokenize_query(query))
    doc_jaccard_scores = {}

    for term in query_tokens:
        if use_champions:
            if term in champion_lists:
                for doc_id in champion_lists[term]:
                    if doc_id not in doc_jaccard_scores.keys():
                        doc_tokens = set([token for token in docs[doc_id]])
                        jaccard_score = len(query_tokens.intersection(doc_tokens)) / len(query_tokens.union(doc_tokens))
                        doc_jaccard_scores[doc_id] = jaccard_score
        else:
            if term in postings:
                for doc_id in postings[term]['posting'].keys():
                    if doc_id not in doc_jaccard_scores.keys():
                        doc_tokens = set([token for token in docs[doc_id]])
                        intersection = len(query_tokens.intersection(doc_tokens))
                        union = len(query_tokens.union(doc_tokens))
                        jaccard_score = intersection / union
                        doc_jaccard_scores[doc_id] = jaccard_score

    sorted_doc_jaccard_scores = sorted(doc_jaccard_scores.items(), key=lambda x: x[1], reverse=True)
    return sorted_doc_jaccard_scores

In [13]:


def calculate_cosine_similarity(query, use_champions=True):
    query_tf, query_sum = calculate_query_tf(query)

    doc_cosine_score_dic = {}

    for token in query_tf.keys():
        if use_champions:
            if token in champion_lists.keys():
                for doc_id in champion_lists[token]:
                    if not doc_id in doc_cosine_score_dic.keys():
                        doc_cosine_score_dic[doc_id] = 0
                    doc_cosine_score_dic[doc_id] += query_tf[token] * postings[token]['tf_idf'][doc_id]

        else:
            if token in postings.keys():
                for doc_id in postings[token]['posting'].keys():
                    if doc_id not in doc_cosine_score_dic.keys():
                        doc_cosine_score_dic[doc_id] = 0
                    doc_cosine_score_dic[doc_id] += query_tf[token] * postings[token]['tf_idf'][doc_id]

    # calculate similarity
    doc_cosine_similarity = {}

    for doc_id in doc_cosine_score_dic.keys():
        doc_cosine_similarity[doc_id] = doc_cosine_score_dic[doc_id] / doc_len[doc_id]
        doc_cosine_similarity[doc_id] = doc_cosine_similarity[doc_id] / query_sum
    sorted_doc_cosine_similarity = sorted(doc_cosine_similarity.items(), key=lambda x: x[1], reverse=True)

    return sorted_doc_cosine_similarity






In [31]:

def print_res(scores):
    counter = 0
    for doc_id, score in scores:
        counter += 1
        if counter > 5:
            break

        content = contents[doc_id]
        print()
        print('DocID: ', doc_id, 'Title:', titles[doc_id], 'URL:', urls[doc_id])
        print('Score: ', score)
        # print('Content:\n ', content)

doc_len = calculate_tf_idf(postings, docs)
champion_lists = generate_champions(postings)

query = "کمیسیون مجلس"
cosine_similarity = calculate_cosine_similarity(query, True)

print_res(cosine_similarity)



DocID:  8274 Title: دستور جلسات هفته آینده کمیسیون‌های مجلس URL: https://www.farsnews.ir/news/14001108000634/دستور-جلسات-هفته-آینده-کمیسیون‌های-مجلس
Score:  0.14766336789653572

DocID:  10084 Title: اعضای کمیسیون فرهنگی به طبس و رفسنجان سفر می‌کنند URL: https://www.farsnews.ir/news/14000916000703/اعضای-کمیسیون-فرهنگی-به-طبس-و-رفسنجان-سفر-می‌کنند
Score:  0.13226377683451065

DocID:  9807 Title: نوروزی، دلخوش و موحد عضو کمیسیون تلفیق بودجه ۱۴۰۱ شدند URL: https://www.farsnews.ir/news/14000923000737/نوروزی-دلخوش-و-موحد-عضو-کمیسیون-تلفیق-بودجه-۱۴۰۱-شدند
Score:  0.13002295817189938

DocID:  9084 Title: کمیسیون تلفیق کلیات لایحه بودجه را تصویب کرد URL: https://www.farsnews.ir/news/14001014001128/کمیسیون-تلفیق-کلیات-لایحه-بودجه-را-تصویب-کرد
Score:  0.12745228375217194

DocID:  11247 Title: تأکید اعضای کمیسیون فرهنگی مجلس بر اصلاح آیین نامه سازمان امور سینمایی URL: https://www.farsnews.ir/news/14000818000811/تأکید-اعضای-کمیسیون-فرهنگی-مجلس-بر-اصلاح-آیین-نامه-سازمان-امور-سینمایی
Score:  0.12501

In [15]:
import pickle


def storeData():
    # Its important to use binary mode
    dbfile = open('examplePickle', 'ab')
    pickle.dump(postings, dbfile)
    dbfile.close()


def loadData():
    dbfile = open('examplePickle', 'rb')
    db = pickle.load(dbfile)
    i = 0
    for keys in db.keys():
        i += 1
        if i == 10:
            break
        print(keys, '=>', db[keys])
    dbfile.close()


print(len(postings.keys()))

storeData()
loadData()


48560
سردار => {'doc_freq': 329, 'posting': {0: [0, 76, 170, 181], 14: [22, 33, 63], 27: [89, 169, 308, 445, 494, 498], 37: [19, 89], 47: [69], 50: [434], 58: [8, 162, 176, 182, 185, 223, 351, 414, 437, 476, 539], 72: [2, 30, 291], 79: [57, 62], 101: [12], 138: [39, 131], 144: [147], 157: [8, 24], 162: [2], 168: [13], 199: [11], 234: [24], 264: [11], 293: [20, 64, 81, 101, 115, 176], 317: [445, 518], 319: [12, 39], 331: [10], 356: [0, 58], 358: [0, 22, 144], 368: [3, 125, 311, 403], 382: [33, 77], 421: [0, 82, 155], 431: [37], 446: [0, 140], 452: [0], 482: [12], 501: [1087, 1094], 504: [194], 508: [10], 516: [24, 286], 567: [7], 568: [397], 574: [13], 592: [124], 631: [658, 709], 642: [16, 44, 100], 662: [660], 668: [25, 41, 65], 689: [224], 703: [20, 159], 714: [0], 727: [12, 38], 728: [0, 254], 767: [36], 774: [0, 21, 50], 802: [11, 63, 224, 300, 344, 471, 552], 813: [241], 839: [1241, 1269], 840: [33], 842: [561], 868: [13, 43, 81, 93, 145, 151, 166], 906: [10], 944: [178, 199], 954